In [1]:
import json
from tqdm import tqdm
with open('../data/chronicle/chronicle_data.json') as f:
    data = json.load(f)

In [2]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="JetBrains-Research/cmg-codet5-with-history")

In [3]:
diffs = []
for commit in data:
    diff = commit['diff']
    diffs.append(diff)

generated_commit_messages = []
for diff in tqdm(diffs[:100], desc='Generating commit messages'):
    prompt = f"""
    The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly.
    {diff}
    According to the diff, the commit message should be:
    """
    generated_commit_messages.append(pipe(prompt)[0]['generated_text'])

Generating commit messages:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/Caskroom/miniconda/base/envs/cmg/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating commit messages: 100%|██████████| 100/100 [02:32<00:00,  1.53s/it]


In [4]:
for item, msg in zip(data[:100], generated_commit_messages):
    item['codet5-history'] = msg

output_file = '../data/chronicle/chronicle_codet5_history.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(data[:100], f, ensure_ascii=False, indent=4)